In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch

### Make sure your transformers >=4.46, we use the latest version of transformers

In [ ]:
# The showcasing model is LLaMa-3-8B-Instruct

model_path = "meta-llama/Llama-3.1-8B-Instruct" # path to your model
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, trust_remote_code=True).cuda().eval()

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.58it/s]


In [3]:
# Below is a sample from XSUM
# The summary term is the ground truth taking for reference
content =  {'document': 'Head of the Army General Sir Nick Carter said the move was about trying to operate "smarter".\nThe 77th Brigade, made up of reservists and regular troops and based in Hermitage, Berkshire, will be formally created in April.\nIt has been inspired by the Chindits who fought in Burma in World War Two.\nAn Army spokesman said the unit would "play a key part in enabling the UK to fight in the information age" and that it "consists of more than just traditional capabilities".\nHe said: "77 Brigade is being created to draw together a host of existing and developing capabilities essential to meet the challenges of modern conflict and warfare.\n"It recognises that the actions of others in a modern battlefield can be affected in ways that are not necessarily violent and it draws heavily on important lessons from our commitments to operations in Afghanistan amongst others."\nRecruitment for the brigade, 42% of whose personnel will be reservists, will begin this spring.\nIts members will come from the Royal Navy and RAF as well as from the Army.\nOne former commander of British forces in Afghanistan has warned the new operation should not mean fewer troops on the frontline.\nColonel Richard Kemp said: "My view is that this should not be done at the expense of combat troops. Where are these 2,000 people going to come from?"\n"They are likely to come from savings made in combat troops. I think that\'s a mistake.\n"I think the British forces have already been cut far too much in a very uncertain and increasingly dangerous world.\nHe acknowledged the need for this type of innovation, but said "it should be added to the forces, not created out of savings found elsewhere."\nThe creation of the new unit is part of a major restructuring of the military under the Army 2020 plan, which will see the military scaled down to around 82,000 regular troops in the next five years.\nThe unit will also seek "new ways of allowing civilians with bespoke skills to serve alongside their military counterparts".\nThe Army spokesman said it would share the "spirit of innovation" of the Chindits in the Burma Campaign of 1942 to 1945.\nChindits was the name given to the Long Range Penetration (LRP) groups that operated in the Burmese jungle behind enemy lines, targeting Japanese communications.\nThe new unit will also use the old Chindit insignia of a Chinthe, a mythical Burmese creature which is half-lion and half-dragon.\nTony Redding from Kent, whose father was in the Chindits, told the BBC he was disappointed by the move. He said: "Sadly the Ministry of Defence didn\'t inform the surviving Chindit veterans of the decision to use the badge in this way.\n"I\'ve tried very hard to look for similarities and the only common denominator I can find is that the Chindits 70 years ago were a highly unconventional force. Perhaps this new force are to use some unconventional means of warfare."\nBy BBC defence correspondent Jonathan Beale\nThe Army says it\'s learnt valuable lessons from Afghanistan - not least that it can\'t win wars using pure military force alone.\nThe brigade will be made up of warriors who don\'t just carry weapons, but who are also skilled in using social media such as Twitter and Facebook, and the dark arts of "psyops" - psychological operations.\nThey will try to influence local populations and change behaviour through what the Army calls traditional and unconventional means.\nCivilians with the right skills will work alongside regular troops and reservists and could be sent anywhere in the world to help win hearts and minds.\nIt can be seen as proof that the Army is adapting to modern asymmetric warfare, and that it remains relevant at a time when there are fears within the British military of more cuts after the election.\nPaul Rogers, a professor of international security at the University of Bradford, said the announcement represented a "big expansion" of the Army\'s psychological operations and was an "attempt to rebrand and update" this area of its work.\n"We had so much difficulty in Iraq and Afghanistan, it\'s about trying to learn the lessons of how these groups are using social media," Prof Rogers explained.\nHe added: "In some senses it\'s defensive - trying to present the case from this side against opponents who hold many of the cards.\n"We\'ve seen with Islamic State, its incredible capability on the net, Facebook, Instagram and all the rest."\nA former Army officer involved in psychological operations in the Balkans, Afghanistan and Iraq, Simon Bergman, predicted it would help build "the Army for the future".\n"For example, 77 brigade have a large component of civil affairs soldiers who\'ll be operating in populations, working with them, achieving military effects - and a broader effect, because as we know from Afghanistan, the military doesn\'t work in isolation. It works as a component of government."',
 'summary': 'The Army is setting up a new unit that will use psychological operations and social media to help fight wars "in the information age".'}

In [4]:
# Prompt Design
messages = [
    {"role": "system", "content": "You are a summarization chatbot who always responds in clear and precise summarization! The summarization should no more than 30 words."},
    {"role": "user", "content": content['document']},
]
input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True,
                                           return_tensors="pt").to(model.device)
terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

## Replace the modeling_llama.py in your environment

In [ ]:
## The quickest way is to make an error, and the warning 
# will guide you to the right place to find modeling_llama.py
outputs = model.generate(input_ids.cpu(),) # Deliberately put to cpu
# after runing this, you will find the modeling_llama.py contains in the warning
# Replace it with the modeling_llama.py provided

### After replacement, Restart the jupyter kernel to let your environment update

### Original Model Output

In [ ]:
# Using SRA, the latest version needs to turn on "output attention" to 
# do explicit attention manipulation, we change defalut configuration to 
# always set output_attention=True but not really output it
outputs = model.generate(
    input_ids.cuda(),
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=False,
    temperature=1.0,
    # top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print("\n", tokenizer.decode(response, skip_special_tokens=True))

## Below is the original output:
# The UK Army is creating the 77th Brigade, a new unit focused on 
# information warfare, psychological operations, and social media influence. 
# It will draw on lessons from Afghanistan and the Chindits, with 42% of personnel being reservists.

## Init SRA

In [9]:
model.model.init_sra(begin_length=5, threshold_len=50,
                     elimi_factor_inter = 0.5, elimi_factor_outer=0.5,
                     strengthen_factor_inter=5, strengthen_factor_outer=5)

Successfully Init SRA


## SRA Output.  Enjoy your own now!

In [ ]:
outputs = model.generate(
    input_ids.cuda(),
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=False,
    temperature=1.0,
    # top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print("\n", tokenizer.decode(response, skip_special_tokens=True))

## SRA Output is:
# The UK Army is creating a new 77th Brigade, inspired by the Chindits, 
# to operate in the "information age" with reservists, 
# regular troops, and civilians, focusing on psychological operations and social media.